<a href="https://colab.research.google.com/github/ARAN1218/UsedcarValuePrediction_AI/blob/main/jupyter%20notebook/7.%E4%B8%AD%E5%8F%A4%E8%BB%8A%E4%BE%A1%E6%A0%BC%E4%BA%88%E6%B8%ACAI(%E4%BE%A1%E6%A0%BC_com)_%E3%82%B5%E3%83%BC%E3%83%93%E3%82%B9%E4%BD%9C%E6%88%90(%E3%83%AD%E3%83%BC%E3%82%AB%E3%83%AB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%run ./UsedCar_functions.ipynb
import_test()

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_pickle('Car_Bigdata_Merged.pickle').drop_duplicates()
print(len(df))
df.head(10)

In [ ]:
kkkcom_preprocessing(df)
print(len(df))
df.head(1)

In [ ]:
print(LE_mission.classes_)
print(LE_fuel.classes_)
print(LE_handle.classes_)
print(LE_usingfuel.classes_)

In [ ]:
#xgboost,lightgbm,ランダムフォレストのアンサンブルで任意データ予測関数作り
def kkkcom_prediction():
    import xgboost as xgb
    import lightgbm as lgb
    import pandas as pd
    import pickle
    from pprint import pprint
    from time import sleep
    
    columns = ['車名', '地域', '年式', '走行距離', '車検', '修復歴'
               , '未使用車', 'ワンオーナー', '禁煙車', 'ミッション', '駆動方式'
               , '排気量', 'ドア数', '乗車定員', '燃料', 'ハンドル', '記録簿'
               , '燃費', '使用燃料', 'エアコン', 'パワーステアリング', 'パワーウィンドウ'
               , 'キーレスエントリー', 'エアバッグ', 'ABS', 'アルミホイール', '盗難防止システム'
               , 'バックモニターカメラ', 'ベンチシート', 'フルフラット', 'CDデッキ', 'カーナビ'
               , 'テレビ', 'ETC', '両側スライドドア両側電動', '両側スライドドア'
               , 'スライドドア電動あり', 'スライドドア', '寒冷地仕様', 'パワーシート'
               , '本革シート', 'サンルーフ', 'ディスチャージドランプ', 'エアロパーツ'
               , '3列シート', 'ウォークスルー', 'ローダウン', '福祉車両', '修復レベル'
               , '保証フラグ', '保証期間', '保証距離'
              ]
    
    #車名と地域を取得する
    with open('Car_LE_name.pickle', mode='rb') as fp:
        LE_n = pickle.load(fp)
    with open('Car_LE_area.pickle', mode='rb') as fp:
        LE_a = pickle.load(fp)
    
    #inputメソッドで任意データを入力する
    #基本編
    pprint(sorted(LE_n.classes_))
    sleep(0.1)
    name = input('上記の選択肢より、車の種類を選んでください>')
    pprint(sorted(LE_a.classes_))
    area = input('上記の選択肢より、地域を入力してください>')
    
    year = int(input('年式を入力してください(ex. 2021)>'))
    distance = int(input('走行距離を入力してください(単位：km)(ex. 10km->10)>'))
    inspection = float(input('次回車検年月と現在の年月の差を入力してください(ex 現在年月が2021年7月で、次回車検年月が2022年9月->202209-202107=102を入力)(車検整備付：400、新車未登録：500、不明：0)>'))
    repair = int(input('修復歴の有無を入力してください(修復歴がある：1、修復歴がない：0)>'))
    unused = int(input('未使用かどうかを入力してください(未使用車である：1、未使用車でない：0)>'))
    oneowner = int(input('ワンオーナー車かどうかを入力してください(ワンオーナー車である：1、ワンオーナー車でない：0)>'))
    nosmoking = int(input('禁煙車かどうかを入力してください(禁煙車である：1、禁煙車でない：0)>'))
    mission = int(input('ミッションを入力してください(AT:1、CVT:2、MT:3)>'))
    drive = int(input('駆動方式を入力してください(2WD:2、4WD:4)>'))
    displacement = int(input('排気量を入力してください(ex. 660cc->660)>'))
    door = int(input('ドア数を入力してください(ex. 4個->4)>'))
    capacity = int(input('乗車定員を入力してください(ex. 4人->4)>'))
    fuel = int(input('燃料を入力してください(その他：0、ガソリン：1、ディーゼル：2)>'))
    handle = int(input('ハンドルを入力してください(右ハンドル：0、左ハンドル：1)>'))
    record = int(input('記録簿の有無を入力してください(有り：1、無し：0)>'))
    fuelconsumption = float(input('燃費を入力してください(単位：km/L)(ex. 27.4km/L->27.4)>'))
    usingfuel = int(input('使用燃料を入力してください(その他：0、ディーゼル：1、ハイオク：2、レギュラー：3)>'))
    repair_lv = int(input('過去の傷の修復レベルを入力してください(無傷：0、軽度：1、中度：2、重度：3)>'))
    is_gua = int(input('保証フラグを入力してください(保証無し：0、有償保証付き：1、無償保証付き：2)>'))
    gua_span = int(input('保証期間を入力してください(単位：ヶ月)(ex. 12ヶ月->12)>'))
    gua_dist = int(input('保証距離を入力してください(単位：km)(ex. 10km->10、無制限->999999)>'))
    
    #オプション編
    print(
        '------------------------------------------------------------------------------------------',
        "\n",
        'ここからはオプションについて入力します。装備がある場合は「1」を、ない場合は「0」を入力してください。',
        "\n",
        '------------------------------------------------------------------------------------------'
    )
    aircon = int(input('エアコンを入力してください>'))
    power_s = int(input('パワーステアリングを入力してください>'))
    power_w = int(input('パワーウィンドウを入力してください>'))
    keyless = int(input('キーレスエントリーを入力してください>'))
    airbag = int(input('エアバッグを入力してください>'))
    absabs = int(input('ABSを入力してください>'))
    al_w = int(input('アルミホイールを入力してください>'))
    nosteal = int(input('盗難防止システムを入力してください>'))
    bmc = int(input('バックモニターカメラを入力してください>'))
    bs = int(input('ベンチシートを入力してください>'))
    ff = int(input('フルフラットを入力してください>'))
    cd = int(input('CDデッキを入力してください>'))
    navi = int(input('カーナビを入力してください>'))
    tv = int(input('テレビを入力してください>'))
    etc = int(input('ETCを入力してください>'))
    bothslide_e = int(input('両側スライドドア(両側電動)を入力してください>'))
    bothslide = int(input('両側スライドドアを入力してください>'))
    slide_e = int(input('スライドドア電動ありを入力してください>'))
    slide = int(input('スライドドアを入力してください>'))
    cold = int(input('寒冷地仕様を入力してください>'))
    power_seat = int(input('パワーシートを入力してください>'))
    leather = int(input('本革シートを入力してください>'))
    sun = int(input('サンルーフを入力してください>'))
    dcl = int(input('ディスチャージドランプを入力してください>'))
    aero = int(input('エアロパーツを入力してください>'))
    seat_3 = int(input('3列シートを入力してください>'))
    walk = int(input('ウォークスルーを入力してください>'))
    low = int(input('ローダウンを入力してください>'))
    welfare = int(input('福祉車両を入力してください>'))

    
    d_try = {
        "車名":name
        ,"年式":year
        ,"走行距離":distance
        ,"地域":area
        ,"車検":inspection
        ,"修復歴":repair
        ,"未使用車":unused
        ,"ワンオーナー":oneowner
        ,"禁煙車":nosmoking
        ,"ミッション":mission
        ,"駆動方式":drive
        ,"排気量":displacement
        ,"ドア数":door
        ,"乗車定員":capacity
        ,"燃料":fuel
        ,"ハンドル":handle
        ,"記録簿":record
        ,"燃費":fuelconsumption
        ,"使用燃料":usingfuel
        
        ,"エアコン":aircon
        ,"パワーステアリング":power_s
        ,"パワーウィンドウ":power_w
        ,"キーレスエントリー":keyless
        ,"エアバッグ":airbag
        ,"ABS":absabs
        ,"アルミホイール":al_w
        ,"盗難防止システム":nosteal
        ,"バックモニターカメラ":bmc
        ,"ベンチシート":bs
        ,"フルフラット":ff
        ,"CDデッキ":cd
        ,"カーナビ":navi
        ,"テレビ":tv
        ,"ETC":etc
        ,"両側スライドドア両側電動":bothslide_e
        ,"両側スライドドア":bothslide
        ,"スライドドア電動あり":slide_e
        ,"スライドドア":slide
        ,"寒冷地仕様":cold
        ,"パワーシート":power_seat
        ,"本革シート":leather
        ,"サンルーフ":sun
        ,"ディスチャージドランプ":dcl
        ,"エアロパーツ":aero
        ,"3列シート":seat_3
        ,"ウォークスルー":walk
        ,"ローダウン":low
        ,"福祉車両":welfare
        ,"修復レベル":repair_lv
        ,"保証フラグ":is_gua
        ,"保証期間":gua_span
        ,"保証距離":gua_dist
    }
    
    #ユーザに入力情報を表示する
    global df_try
    df_try = pd.DataFrame(d_try, index=['own'])
    display(df_try)
    
    #入力したデータを予測に使えるように整形する
    df_try['車名'] = LE_name.transform(df_try['車名'])
    df_try['地域'] = LE_area.transform(df_try['地域'])
    merge_preprocessing(df_try)
    display(df_try)
    
    #予め保存しておいたモデルを読み込む
    with open('Carmodel_xgb.pickle', mode='rb') as fp:
        Carmodel_xgb = pickle.load(fp)
    with open('Carmodel_lgb.pickle', mode='rb') as fp:
        Carmodel_lgb = pickle.load(fp)

    #種々のモデルにデータを入力して予測させる
    pred_xgb = Carmodel_xgb.predict(xgb.DMatrix(df_try))
    pred_lgb = Carmodel_lgb.predict(df_try)
    
    #平均をとる・加重平均をとる等の方法で最終的な予測値を決定する
    pred_final = (pred_xgb + pred_lgb) / 2
    
    #体裁を整えて予測結果を出力する
    print('あなたの車の売値相場は',pred_final,'万円です！')


In [ ]:
kkkcom_prediction()

In [ ]:
df[df['車名']==2805]